In [1]:
!pip3 install torch torchvision 

  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached MarkupSafe-3.0.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (4.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 MB 11.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 20.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 13.2 MB/s eta 0:00:00
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
Using cached MarkupSafe-3.0.2-cp313-cp313-macosx_11_0_arm64.whl (12 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [torchvision] [torchvision]


In [2]:
# 1. 导入所需要的库
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms



In [3]:
# 2. 下载CIFAR-10数据集
# 设置图像预处理: 图像增强 + 转换为张量 + 标准化
transform = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
     transforms.RandomCrop(32, padding=4),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# 下载训练集和测试集
print("[LOG] Now loading CIFAR-10 dataset for Training...")
trainset = torchvision.datasets.CIFAR10(root='CIFAR10', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)
print("[LOG] Now loading CIFAR-10 dataset for Testing...")
testset = torchvision.datasets.CIFAR10(root='CIFAR10', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)
print("[LOG] Loading CIFAR-10 dataset finished.")

[LOG] Now loading CIFAR-10 dataset for Training...
[LOG] Now loading CIFAR-10 dataset for Testing...
[LOG] Loading CIFAR-10 dataset finished.


In [4]:
# 3. 使用ResNet-18作为预训练网络
# 下载预训练的ResNet-18模型
print("[LOG] Now loading model RestNet-18...")
resnet18 = torchvision.models.resnet18(pretrained=True)
print("[LOG] Loading model ResNet-18 finished.")
# 由于CIFAR-10有10个类，我们需要调整ResNet的最后一个全连接层
num_classes = 10
resnet18.fc = nn.Linear(resnet18.fc.in_features, num_classes)

[LOG] Now loading model RestNet-18...
[LOG] Loading model ResNet-18 finished.


/opt/homebrew/Caskroom/miniconda/base/envs/ml/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/ml/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
import torch.mps
import torch.xpu


device = 'cpu'
if(torch.cuda.is_available()):
    device = 'cuda:0'
if(torch.xpu.is_available()):
    device = 'xpu'
if(torch.mps.is_available()):
    device = 'mps'
print("[LOG] Using device: ", device)

[LOG] Using device:  mps


In [6]:
# 4. 微调预训练的CNN网络
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet18.parameters(), lr=0.001, momentum=0.9)



In [7]:
# 迁移到GPU上（如果有的话）
resnet18.to(device)
# 训练网络
print("[LOG] Start training...")
for epoch in range(10):  # 就演示训练10个epochs
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # 获取输入数据
        inputs, labels = data[0].to(device), data[1].to(device)
        # 清零参数梯度
        optimizer.zero_grad()
        # 前向 + 反向 + 优化
        outputs = resnet18(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 200 == 199:  # 每200批次打印一次
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0
print('[LOG] Training finished')

[LOG] Start training...
[1,   200] loss: 1.592
[1,   400] loss: 1.147
[1,   600] loss: 1.032
[2,   200] loss: 0.864
[2,   400] loss: 0.848
[2,   600] loss: 0.801
[3,   200] loss: 0.727
[3,   400] loss: 0.725
[3,   600] loss: 0.711
[4,   200] loss: 0.667
[4,   400] loss: 0.655
[4,   600] loss: 0.626
[5,   200] loss: 0.600
[5,   400] loss: 0.604
[5,   600] loss: 0.588
[6,   200] loss: 0.559
[6,   400] loss: 0.548
[6,   600] loss: 0.574
[7,   200] loss: 0.525
[7,   400] loss: 0.531
[7,   600] loss: 0.532
[8,   200] loss: 0.514
[8,   400] loss: 0.493
[8,   600] loss: 0.501
[9,   200] loss: 0.472
[9,   400] loss: 0.471
[9,   600] loss: 0.485
[10,   200] loss: 0.442
[10,   400] loss: 0.456
[10,   600] loss: 0.444
[LOG] Training finished


In [8]:
# 5. 测试网络性能
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = resnet18(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('[LOG] Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

[LOG] Accuracy of the network on the 10000 test images: 80 %
